# Differential Geometry #
learning from the book *An Introduction to Riemannian Geometry* by Leonor Godinho and José Natário.

## topological manifold ##
a topological manifold $M$ of dimension $n$ is a topological space with the following properties

Primary
- Each point $p \in M$ has a neborhood homeomorphic to an open subset of $\mathbb{R}^n$

Nessisary to maintain sanity
- $M$ is **Hausdorff** space
- $M$ is a **second-countable** space

**second-countable** space: is a topological space whose topology has a countable base

**Hausdorff** space: is a topological space where for any two distinct points there exist neighbourhoods of each which are disjoint from each other.


In [ ]:
import Test.QuickCheck

In [ ]:
class LinearSpace a where
    vectorAdd :: a -> a -> a
    scalarMult :: Double -> a -> a
    vectorZero :: a

class MetricSpace a where
    distanceSquared :: a -> a -> Double
    findBoundingBox :: [a] -> (a, Double)

class Domain a where
    inDomain :: (LinearSpace s, MetricSpace s) =>  a s -> s -> Bool

class Homeomorphism a where
    move :: a n m -> a n m -> n -> n

In [ ]:
--data Point = R2 Double Double | R3 Double Double Double deriving Show -- | Rn Integer [Double] 
data R2 = R2 Double Double deriving Show
data R3 = R3 Double Double Double deriving Show

type Curve a = [a]

data Region a = Square a Double | Disk a Double | Annulus a Double Double deriving (Eq, Show)

data Map a b = Map {parameterization::a -> b, coordinateSystem::b -> a}

data Chart a b = Chart (Region a) (Map a b)

data ChartCurve a b = ChartCurve (Curve a) (Chart a b)

type Manifold a b = [Chart a b]


## Graphing basics ##

In [ ]:
import Graphics.Rendering.Chart.Easy
import Graphics.Rendering.Chart.Grid
import Graphics.Rendering.Chart.Backend.Cairo

linspace :: Double -> Double -> Int -> [Double]
linspace start stop num =
    let dx = (stop - start)/fromIntegral (num - 1)
    in [fromIntegral i*dx + start| i <- [0 .. (num - 1)]]

lerp :: LinearSpace a => Double -> a -> a -> a
lerp t p_0 p_1 = vectorAdd (scalarMult (1-t) p_0) (scalarMult t p_1)

lerp' :: LinearSpace a => a -> a -> a
lerp' p_0 p_1 = vectorAdd p_1 (scalarMult (-1) p_0)

bezierRedux :: (LinearSpace a) => Double -> [a] -> a
bezierRedux _ [] = error "One or more points must be given"
bezierRedux _ [point] = point
bezierRedux t points = bezierRedux t [ lerp t p1 p2 | (p1, p2) <- zip (init points) (tail points)]

bezierRedux' :: (LinearSpace a) => Double -> [a] -> a
--bezierRedux' _ [] = error "One or more points must be given"
bezierRedux' _ [point] = vectorZero
bezierRedux' t points = vectorAdd (bezierRedux' t [ lerp t p1 p2 | (p1, p2) <- zip (init points) (tail points)]) (bezierRedux t [ lerp' p1 p2 | (p1, p2) <- zip (init points) (tail points)])

bezierCurveClosed :: (LinearSpace a) => Int -> [a] -> Curve a
bezierCurveClosed resolution points = [bezierRedux t (last points:points) | t <- linspace 0 1 resolution]

bezierCurve :: (LinearSpace a) => Int -> [a] -> Curve a
bezierCurve resolution points = [bezierRedux t points | t <- linspace 0 1 resolution]

--derivative of the curve
bezierCurve' :: (LinearSpace a) => Int -> [a] -> Curve a
bezierCurve' resolution points = [bezierRedux' t points | t <- linspace 0 1 resolution]

curveFilterBoundary :: (Domain a, LinearSpace b, MetricSpace b) => a b -> Curve b -> ([Curve b], [Curve b])
curveFilterBoundary region curve = (filter (inDomain region . head) curves, filter ((not . inDomain region) . head) curves)
    where curves = curveSplitOnBoundary region curve

curveSplitOnBoundary :: (Domain a, LinearSpace b, MetricSpace b) => a b -> Curve b -> [Curve b]
curveSplitOnBoundary region [] = []
curveSplitOnBoundary region curve = c:curveSplitOnBoundary region cs
    where
    c = curveTakeFirstBoundary region curve
    cs = drop (length c) curve

curveTakeFirstBoundary :: (Domain a, LinearSpace b, MetricSpace b) => a b -> Curve b -> Curve b
curveTakeFirstBoundary _ [point] = [point]
curveTakeFirstBoundary region (p1:p2:other) | inDomain region p1 == inDomain region p2 = p1:curveTakeFirstBoundary region (p2:other)
                                            | otherwise = [p1]

binding :: (Monad m) => [m a] -> m a
binding [a] = a
binding (a:as) = a >> binding as

plotSideBySide plot1 plot2 = aboveN [ besideN [ layoutToGrid (execEC plot1), layoutToGrid (execEC plot2)]]
plotNSideBySide plots = aboveN [ besideN [ layoutToGrid (execEC plot) | plot <- plots]]

plotR2Points pointlist = plot (points "point in R2" [(x, y) | (R2 x y) <- pointlist])
plotR2Line pointlist = plot (line "line in R2" [[(x, y) | (R2 x y) <- pointlist]])
plotR2Lines lineList = plot (line "lines in R2" [[(x, y) | (R2 x y) <- pointList] | pointList <- lineList])

plotCurveTransformed func curve = plotCurve [func point | point <- curve]
plotCurvesTransformed func curves = plotCurves [[func point | point <- curve] | curve <- curves]

plotCurve = plotR2Line 
plotCurves = plotR2Lines

--plotTaylorSeries functions x = toRenderable $ binding [plot (line name [eval function x]) | (function, name) <- functions]

## R2 and R3 ##

In [ ]:
(~=) :: (Num a, Ord a, Fractional a) => a -> a -> Bool
(~=) a b = abs(a - b) < 1e-5

instance MetricSpace R2 where
    distanceSquared (R2 x1 y1) (R2 x2 y2) = (x1 - x2)^2 + (y1 - y2)^2
    findBoundingBox points = (R2 cx cy, maximum [rx, ry]) where
        (xs, ys) = ([x | (R2 x _) <- points], [y | (R2 _ y) <- points])
        (xmax, xmin) = (maximum xs, minimum xs)
        (ymax, ymin) = (maximum ys, minimum ys)
        (cx, cy) = ((xmax + xmin)/2, (ymax + ymin)/2)
        (rx, ry) = ((xmax - xmin)/2, (ymax - ymin)/2)
    
instance MetricSpace R3 where
    distanceSquared (R3 x1 y1 z1) (R3 x2 y2 z2) = (x1 - x2)^2 + (y1 - y2)^2 + (z1 - z2)^2
    findBoundingBox points = (R3 cx cy cz, maximum [rx, ry, rz]) where
        (xs, ys, zs) = ([x | (R3 x _ _) <- points], [y | (R3 _ y _) <- points], [z | (R3 _ _ z) <- points])
        (xmax, xmin) = (maximum xs, minimum xs)
        (ymax, ymin) = (maximum ys, minimum ys)
        (zmax, zmin) = (maximum zs, minimum zs)
        (cx, cy, cz) = ((xmax + xmin)/2, (ymax + ymin)/2, (zmax + zmin)/2)
        (rx, ry, rz) = ((xmax - xmin)/2, (ymax - ymin)/2, (zmax - zmin)/2)

instance LinearSpace R2 where
    vectorAdd (R2 x1 y1) (R2 x2 y2) = R2 (x1 + x2) (y1 + y2)
    scalarMult s (R2 x y) = R2 (s*x) (s*y)
    vectorZero = R2 0 0
    
instance LinearSpace R3 where
    vectorAdd (R3 x1 y1 z1) (R3 x2 y2 z2) = R3 (x1 + x2) (y1 + y2) (z1 + z2)
    scalarMult s (R3 x y z) = R3 (s*x) (s*y) (s*z)
    vectorZero = R3 0 0 0

instance Eq R2 where
    (==) (R2 a b) (R2 c d) = (a ~= c) && (b ~= d)

instance Eq R3 where
    (==) (R3 a b c) (R3 d f g) = (a ~= d) && (b ~= f) && (c ~= g)

-- for generating arbitrary elements of R2 and R3

instance Arbitrary R2 where
    arbitrary = do
        a <- arbitrary
        b <- arbitrary
--        return (R2 (a/10) (b/10))
        return (R2 (a) (b))

instance Arbitrary R3 where
    arbitrary = do
        a <- arbitrary
        b <- arbitrary
        c <- arbitrary
        return (R3 (a/10) (b/10) (c/10))

genR2 :: Gen R2
genR2 = arbitrary

genR2' :: Gen R2
genR2' = do
        a <- arbitrary
        b <- arbitrary
        return (R2 a b)

genR3 :: Gen R3
genR3 = arbitrary

## Region ##
Regions are defined on R2 or R3

In [ ]:
instance Domain Region where
    inDomain (Square center halfWidth) point = 2*boxHalfWidth < halfWidth
        where (_, boxHalfWidth) = findBoundingBox [center, point]
    inDomain (Disk center r) point = (distanceSquared center point) < r^2
    inDomain (Annulus center r1 r2) point = ((distanceSquared center point) > r1^2) && ((distanceSquared center point) < r2^2)

_boundaryRes = 200

boundary :: Region R2 -> [[R2]]
boundary (Square (R2 x y) hd) = [concat [[lerp t corner1 corner2 | t <- linspace 0 1 _boundaryRes] | (corner1, corner2) <- zip (init corners) (tail corners)]]
    where corners = [R2 (x - hd) (y - hd), R2 (x - hd) (y + hd), R2 (x + hd) (y + hd), R2 (x + hd) (y - hd), R2 (x - hd) (y - hd)]
boundary (Disk (R2 x y) r) = [[R2 (x + r*sin t) (y + r*cos t) | t <- linspace 0 (2*pi) (_boundaryRes*4)]]
boundary (Annulus (R2 x y) r1 r2) = [[R2 (x + r1*sin t) (y + r1*cos t) | t <- linspace 0 (2*pi) (_boundaryRes*2)], [R2 (x + r2*sin t) (y + r2*cos t) | t <- linspace 0 (2*pi) (_boundaryRes*2)]]

-- for generating arbitrary regions

genPos :: Gen Double
genPos = abs `fmap` (arbitrary :: Gen Double) `suchThat` (> 0)

arbitrarySquareRegion = do
    point <- arbitrary
    Square point <$> genPos
    
arbitraryDiskRegion = do
    point <- arbitrary
    Disk point <$> genPos

ord (a, b) | b < a = (b, a)
           | otherwise = (a, b)

arbitraryAnnulusRegion = do
    point <- arbitrary
    a <- genPos
    b <- genPos
    let (r1, r2) = ord (a, b)
    return (Annulus point r1 r2)

instance (Arbitrary a) => Arbitrary (Region a) where
    arbitrary = oneof [arbitrarySquareRegion, arbitraryDiskRegion, arbitraryAnnulusRegion]
    
genRegionR2 :: Gen (Region R2)
genRegionR2 = arbitrary

genRegionR3 :: Gen (Region R3)
genRegionR3 = arbitrary

-- Testing

prop a = a === a where
    types = a::(Region R2)

propAnn (Annulus pnt r1 r2) = r1 < r2
    where types = pnt::R3
    
quickCheck $ forAll arbitraryAnnulusRegion propAnn
quickCheck prop

## Ploting with points and regions ##

In [ ]:
--generate genR2
pointlist = generate (listOf genR2')

plotR2Boundary = plotR2Lines . boundary
-- generate (boundary <$> arbitrarySquareRegion)

plotR2Boundarys r1 r2 = plotR2Boundary r1 >> plotR2Boundary r2

plotR2BoundaryAndPoints region points = plotR2Boundary region >> plotR2Points pl >> plotR2Points pl'
    where
    pl = filter (inDomain region) points
    pl' = filter (not . inDomain region) points

--ppp d = (plotR2 d) >> (plotR2' d)
--ppp' d b = (plotR2 d) >> (plotR2' b)

boundingBoxOfPoints points = plotR2Boundary region
    where
    (center, r) = findBoundingBox points
    region = Square center r

--toRenderable $ plotR2BoundaryAndPoints (Square (R2 0 0) 0.5) [R2 0 2, R2 1.5 1, R2 (-1) 0.5]
--toRenderable <$> (plotR2BoundaryAndPoints <$> (generate genRegionR2) <*> pointlist)
toRenderable <$> (plotR2BoundaryAndPoints <$> (generate arbitrarySquareRegion) <*> pointlist)

--toRenderable . plotR2Lines <$> generate (boundary <$> genRegionR2)
--toRenderable . plotR2Boundary <$> generate genRegionR2
--toRenderable <$> (plotR2Boundarys <$> generate genRegionR2 <*> generate genRegionR2)

--toRenderable . plotR2Boundary <$> generate (boundary <$> arbitraryDiskRegion)
--toRenderable . plotR2Boundary <$> generate (boundary <$> arbitraryAnnulusRegion)
--toRenderable <$> (plotR2 <$> pointlist)
--toRenderable <$> (plotR2' <$> pointlist)
--toRenderable <$> (ppp <$> pointlist)
--toRenderable <$> (ppp' <$> pointlist <*> pointlist)


## Plotting with curves and regions ##

In [ ]:
plotBezier contpoints = plotR2Line points >> boundingBoxOfPoints points
    where points = [bezierRedux t contpoints | t <- linspace 0 1 1000]
plotBezVerbose initpoints = plotBezier points >> plotR2Line points
    where points = take 5 initpoints

plotTestDeriv contpoints = plotCurve (bezierCurve 100 contpoints) >> plotCurve (bezierCurve' 100 contpoints)

--layout_y_axis . laxis_generate .= scaledAxis def (-10,80)

plotTestDeriv3 contpoints = do
    --layout_x_axis . laxis_generate .= scaledAxis def (-30,30)
    --layout_y_axis . laxis_generate .= scaledAxis def (-15,15)
    plotCurve points
    plotCurves lineSegmentsOrth
        where
        --lineSegments = [[(R2 px py), (R2 (px + 0.01*tx) (py + 0.01*ty))] | ((R2 px py), (R2 tx ty)) <- zip points tangents]
        lineSegmentsOrth = [[(R2 px py), (R2 (px + ty/sqrt (ty*ty + tx*tx)) (py - tx/sqrt (ty*ty + tx*tx)))] | ((R2 px py), (R2 tx ty)) <- zip points tangents]
        points = (bezierCurve 100 contpoints)
        tangents = (bezierCurve' 100 contpoints)

plotTestDeriv2 contpoints = plotCurves lineSegments >> plotCurves lineSegmentsOrth
    where
    lineSegments = [[(R2 px py), (R2 (px + 0.01*tx) (py + 0.01*ty))] | ((R2 px py), (R2 tx ty)) <- zip points tangents]
    lineSegmentsOrth = [[(R2 px py), (R2 (px + 0.01*ty) (py - 0.01*tx))] | ((R2 px py), (R2 tx ty)) <- zip points tangents]
    points = (bezierCurve 100 contpoints)
    tangents = (bezierCurve' 100 contpoints)

(toRenderable . plotTestDeriv3) <$> pointlist
--toRenderable $ plotCurve (bezierCurve 100 [R2 0 0, R2 0.25 0.5, R2 0.75 0.5, R2 1 0])
--toRenderable $ plotCurve (bezierCurve' 100 [R2 0 0, R2 0.25 0.5, R2 0.75 0.5, R2 1 0])
--toRenderable . plotBezVerbose <$> pointlist

plotR2BoundaryAndCurve region curve = plotR2Boundary region >> plotCurves curvesIn >> plotCurves curvesOut
    where (curvesIn, curvesOut) = curveFilterBoundary region curve

toRenderable <$> (plotR2BoundaryAndCurve <$> generate genRegionR2 <*> (bezierCurve 1000 <$> pointlist))

--toRenderable . plotCurve . (bezierCurve 100) <$> pointlist
--toRenderable $ plotR2Line [bezierRedux t [a, b, R2 0.5 2, c] | t <- linspace 0 1 10]
--toRenderable $ plotR2Lines [[lerp t a b | t <- linspace 0 1 10], [lerp t b c | t <- linspace 0 1 10]]
--toRenderable $ plotR2Line [quad t a b c | t <- linspace 0 1 100]

--[lerp t a b | t <- linspace 0 1 10]

## Plotting Under Transformations ##

In [ ]:
testFunc (R2 a b) = R2 (a*10/(a*a + b*b)) (b*10/(b*b + a*a))

plotDomainAndRange func region = plotSideBySide (plotR2Boundary region >> plotR2Points [center]) (plotCurvesTransformed func (boundary region) >> plotR2Points [func center])
    where
    (center, r) = (findBoundingBox . head) $ boundary region

toRenderable . (plotDomainAndRange testFunc) <$> generate genRegionR2


plotR2BoundaryAndCurveDomainRange func region curve = plotSideBySide (plotCurves boundaryCurve >> plotCurves curvesIn >> plotCurves curvesOut >> plotR2Points [center])
                                                                     (plotCurvesTransformed func boundaryCurve >> plotCurvesTransformed func curvesIn >> plotCurvesTransformed func curvesOut >> plotR2Points [func center])
    where
    (curvesIn, curvesOut) = curveFilterBoundary region curve
    boundaryCurve = boundary region
    (center, r) = (findBoundingBox . head) $ boundaryCurve

--(a, b, c) = (R2 0 0, R2 2 2, R2 4 0)
--toRenderable $ plotR2BoundaryAndCurveDomainRange testFunc (Square (R2 0 0) 1) (bezierCurve 100 [a, b, c])
toRenderable <$> ((plotR2BoundaryAndCurveDomainRange testFunc) <$> generate genRegionR2 <*> (bezierCurve 1000 <$> pointlist))
toRenderable <$> ((plotR2BoundaryAndCurveDomainRange testFunc) <$> generate genRegionR2 <*> (bezierCurveClosed 1000 <$> pointlist))
--toRenderable <$> (plotR2BoundaryAndCurve <$> generate genRegionR2 <*> (bezierCurve 1000 <$> pointlist))

## Manifolds ##
validation of transforms

In [ ]:
instance Homeomorphism Map where
    move map1 map2 = (coordinateSystem map2) . (parameterization map1)

foo1 (R2 a b) = R3 x y z
    where
     factor = a*a + b*b + 1
     (x, y, z) = (2*a/factor, 2*b/factor, (a*a + b*b - 1)/factor)
foo1Inv (R3 a b c) = R2 x y
    where
     lambda = 1 - c
     (x, y) = (a/lambda, b/lambda)

foo2 (R2 a b) = R3 x y z
    where
     factor = a*a + b*b + 1
     (x, y, z) = (2*a/factor, 2*b/factor, (1 - a*a - b*b)/factor)
foo2Inv (R3 a b c) = R2 x y
    where
     lambda = 1 + c
     (x, y) = (a/lambda, b/lambda)

-- for testing

propfb1 (R2 a b) = (foo1Inv . foo1) (R2 a b) === R2 a b
propfb2 (R2 a b) = (foo2Inv . foo2) (R2 a b) === R2 a b
propf1OnShell (R2 a b) = distanceSquared (foo1 (R2 a b)) (R3 0 0 0) ~= 1
propf2OnShell (R2 a b) = distanceSquared (foo2 (R2 a b)) (R3 0 0 0) ~= 1
propf1Ring theta = z ~= 0.0
    where
    point = R2 (cos theta) (sin theta)
    (R3 x y z) = foo1 point
propf2Ring theta = z ~= 0.0
    where
    point = R2 (cos theta) (sin theta)
    (R3 x y z) = foo2 point

test = id -- . verbose

--quickCheck propM
print "test foo1"
(quickCheck . test) propfb1
print "test foo2"
(quickCheck . test) propfb2
print "foo1 on shell"
(quickCheck . test) propf1OnShell
print "foo2 on shell"
(quickCheck . test) propf1OnShell
print "foo1 ring on plane"
quickCheck propf1Ring
print "foo2 ring on plane"
quickCheck propf2Ring

## Plotting on Manifold ##

In [ ]:
map1 = Map foo1 foo1Inv
map2 = Map foo2 foo2Inv
chart1 = Chart (Disk (R2 0 0) 10) map1
chart2 = Chart (Disk (R2 0 0) (1.0/5)) map2
manifold = [chart1, chart2]::(Manifold R2 R3)


-- data Chart a b = Chart (Region a) (Map a b)
-- data ChartCurve a b = ChartCurve (Curve a) (Chart a b)

prop point =
    not (distanceSquared point (R2 0 0) ~= 0.0) ==>
    move map2 map1 (move map1 map2 point) === point
quickCheck prop

plotManifoldChartDomains manifold = plotNSideBySide [plotCurves boundaryCurves | boundaryCurves <- boundaryCurvesRegions]
    where
    boundaryCurvesRegions = [boundary region | (Chart region chartMap) <- manifold]

toRenderable $ plotManifoldChartDomains manifold

plotManifoldCharts manifold = plotNSideBySide [plotCurves boundaryCurves | boundaryCurves <- boundaryCurvesRegions]
    where
    boundaryCurvesRegions = [concat [ moveCurves fromChartMap chartMap (boundary fromRegion) | (Chart fromRegion fromChartMap) <- manifold] | (Chart region chartMap) <- manifold]
    
    moveCurve targetMap destMap curve = [move targetMap destMap point | point <- curve]
    moveCurves targetMap destMap curves = [moveCurve targetMap destMap curve | curve <- curves]

toRenderable $ plotManifoldCharts manifold



plotManifoldCurve manifold chartCurve = plotNSideBySide [plotCurves boundaryCurves >> plotCurve curveOnRegion | (boundaryCurves, curveOnRegion) <- zip boundaryCurvesRegions curveOnRegions]
    where
    (ChartCurve originalCurve (Chart originalCurveRegion originalCurveMap)) = chartCurve
    boundaryCurvesRegions = [concat [ moveCurves fromChartMap chartMap (boundary fromRegion) | (Chart fromRegion fromChartMap) <- manifold] | (Chart region chartMap) <- manifold]
    curveOnRegions = [moveCurve originalCurveMap chartMap originalCurve | (Chart region chartMap) <- manifold]
    
    moveCurve targetMap destMap curve = [move targetMap destMap point | point <- curve]
    moveCurves targetMap destMap curves = [moveCurve targetMap destMap curve | curve <- curves]

testplot points =  plotManifoldCurve manifold (ChartCurve (bezierCurveClosed 1000 points) chart1)
toRenderable <$> testplot <$> pointlist